# Summary

`# TODO: summary here`

In [1]:
%load_ext autoreload
%autoreload 3

In [4]:
import gc
import inspect
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path

from htools import *

In [3]:
cd_root()

Current directory: /Users/hmamin/incendio


/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
print(
"""
# OPTION 1
@auto_cli()
def main():
    x = f1(a, b, c)
    x = f2(x=x, y=y, **f2_kwargs)
    print(x)

if __name__ == '__main__':
    fire.Fire(main)
    
    
# OPTION 2
@auto_cli(f1, f2)
def main():
    x = f1(a, b, c)
    x = f2(x=x, y=y, **f2_kwargs)
    print(x)

if __name__ == '__main__':
    fire.Fire(main)
"""
)


# OPTION 1
@auto_cli()
def main():
    x = f1(a, b, c)
    x = f2(x=x, y=y, **f2_kwargs)
    print(x)

if __name__ == '__main__':
    fire.Fire(main)
    
    
# OPTION 2
@auto_cli(f1, f2)
def main():
    x = f1(a, b, c)
    x = f2(x=x, y=y, **f2_kwargs)
    print(x)

if __name__ == '__main__':
    fire.Fire(main)

